In [1]:
import pandas as pd
import numpy as np
import time
import fpGrowth 

In [2]:
#预处理数据
def pre_data(df):
    df['timestamp'] = pd.to_datetime(df['sldat'])
    
    #分别按vipno、sldat排序
    data = df.sort_values(by=['vipno', 'sldat'])
    
    #分组
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0, method='first')
    
    #取每个vipno的前60%的数据
    data = data.groupby(['vipno'], as_index = False).apply(lambda x: x[x['rank'] <=  round(0.6 * x['rank'].max())])
    #整理
    data = data.drop(['timestamp', 'rank'], axis = 1).reset_index(drop=True)
    return data

In [3]:
#提出bndno的数据
def pre_bndno(data):
    #丢弃多余的列
    data_bndno = data.drop(['sldat', 'pluno', 'dptno'], axis = 1)
    
    #合并订单
    data_bndno['value'] = data_bndno['bndno']
    data_bndno = data_bndno.pivot_table(data_bndno , index = ['vipno'], columns = 'bndno')
    
    #整理
    data_bndno = data_bndno.fillna(0).transpose().loc['value'].transpose()
    del data_bndno.index.name
    del data_bndno.transpose().index.name
    
    #将dataframe转为array
    array_bndno = []
    for row in data_bndno.as_matrix():
        array_bndno.append([x for x in row if x != 0.0])
    return array_bndno

In [4]:
def fp_bndno(data):
    thresholds = [2, 4, 6, 8, 10]
    array_bndno= pre_bndno(data)
    for n in thresholds:
        freq_sets = fpGrowth.fpGrowth(array_bndno, n)
        print("for dptno, threshold: ", n)
        for k in freq_sets:
            if len(k) >= 2:
                print(k, freq_sets[k])

In [5]:
if __name__ == "__main__":
    old_df = pd.read_csv('../trade.csv', usecols=['vipno', 'sldat', 'pluno', 'dptno', 'bndno'])
    #旧数据
    old_data = pre_data(old_df)
    start = time.clock()
    fp_bndno(old_data)
    time_old = time.clock() - start
    
    #新数据
    new_df = pd.read_csv('../trade_new.csv', usecols=['vipno', 'sldatime', 'pluno', 'dptno', 'bndno'])
    new_df.rename(columns={ new_df.columns[0]: "sldat" }, inplace=True)    
    new_data = pre_data(new_df)
    start = time.clock()
    fp_bndno(new_data)
    time_new = time.clock() - start
    
    print("for bndno, old_data:" , time_old)
    print("for bndno, new_data:" , time_new)

for dptno, threshold:  2
frozenset({30248.0, 14612.0}) 2
frozenset({14612.0, 15039.0}) 2
frozenset({30248.0, 14612.0, 15039.0}) 2
frozenset({14612.0, 10060.0}) 2
frozenset({30248.0, 14612.0, 10060.0}) 2
frozenset({14612.0, 10068.0}) 2
frozenset({30248.0, 14612.0, 10068.0}) 2
frozenset({14612.0, 10060.0, 10068.0}) 2
frozenset({30248.0, 14612.0, 10060.0, 10068.0}) 2
frozenset({14281.0, 14445.0}) 2
frozenset({14177.0, 14445.0}) 2
frozenset({14442.0, 14445.0}) 2
frozenset({14314.0, 15012.0}) 2
frozenset({14314.0, 15052.0}) 2
frozenset({15012.0, 14314.0, 15052.0}) 2
frozenset({14314.0, 14319.0}) 2
frozenset({14314.0, 15012.0, 14319.0}) 2
frozenset({14314.0, 15052.0, 14319.0}) 2
frozenset({15012.0, 14314.0, 15052.0, 14319.0}) 2
frozenset({14314.0, 15094.0}) 2
frozenset({14314.0, 15012.0, 15094.0}) 2
frozenset({14314.0, 15052.0, 15094.0}) 2
frozenset({15012.0, 14314.0, 15052.0, 15094.0}) 2
frozenset({14314.0, 15094.0, 14319.0}) 2
frozenset({14314.0, 15012.0, 15094.0, 14319.0}) 2
frozenset({14

frozenset({10086.0, 30248.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 30248.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14338.0, 10086.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14338.0, 10086.0, 30248.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 14338.0, 10086.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 14338.0, 10086.0, 30248.0, 15052.0, 14333.0, 15039.0}) 2
frozenset({15094.0, 14333.0, 10086.0, 15039.0}) 2
frozenset({10086.0, 15094.0, 30248.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 15094.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 15094.0, 30248.0, 14333.0, 15039.0}) 2
frozenset({14338.0, 10086.0, 15094.0, 14333.0, 15039.0}) 2
frozenset({14338.0, 10086.0, 15094.0, 30248.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 14338.0, 10086.0, 15094.0, 14333.0, 15039.0}) 2
frozenset({14177.0, 14338.0, 10086.0, 15094.0, 30248.0, 14333.0, 15039.0}) 2
frozenset({10086.0, 15094.0, 15052

frozenset({30248.0, 10086.0, 15094.0, 10094.0}) 2
frozenset({10086.0, 10094.0}) 3
frozenset({30248.0, 10086.0, 10094.0}) 3
frozenset({30248.0, 10094.0}) 5
frozenset({14442.0, 14372.0}) 2
frozenset({30248.0, 14442.0, 14372.0}) 2
frozenset({14168.0, 14372.0}) 2
frozenset({15052.0, 14372.0}) 2
frozenset({14372.0, 15094.0}) 2
frozenset({15052.0, 14372.0, 15094.0}) 2
frozenset({15011.0, 14372.0}) 2
frozenset({14372.0, 14071.0}) 2
frozenset({14281.0, 14372.0, 14071.0}) 2
frozenset({30248.0, 14372.0}) 3
frozenset({14281.0, 14372.0}) 3
frozenset({30248.0, 14281.0, 14372.0}) 2
frozenset({15057.0, 14098.0}) 2
frozenset({15057.0, 15012.0}) 2
frozenset({15057.0, 15052.0, 15012.0}) 2
frozenset({15057.0, 15012.0, 11079.0}) 2
frozenset({15057.0, 15052.0, 15012.0, 11079.0}) 2
frozenset({15057.0, 15094.0}) 2
frozenset({15057.0, 15052.0, 15094.0}) 2
frozenset({15057.0, 15012.0, 15094.0}) 2
frozenset({15057.0, 15052.0, 15012.0, 15094.0}) 2
frozenset({15057.0, 15094.0, 11079.0}) 2
frozenset({15057.0, 1505

frozenset({10086.0, 30248.0, 15066.0, 10060.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 15066.0, 10060.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 30248.0, 15066.0, 10060.0, 15039.0}) 2
frozenset({10086.0, 10060.0, 15066.0, 15052.0, 15039.0}) 2
frozenset({10086.0, 10060.0, 30248.0, 15066.0, 15052.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 10060.0, 15066.0, 15052.0, 15039.0}) 2
frozenset({14177.0, 10086.0, 10060.0, 30248.0, 15066.0, 15052.0, 15039.0}) 2
frozenset({15066.0, 10060.0, 15094.0}) 2
frozenset({14177.0, 15066.0, 10060.0, 15094.0}) 2
frozenset({15066.0, 10060.0, 10086.0, 15094.0}) 2
frozenset({14177.0, 10086.0, 15094.0, 15066.0, 10060.0}) 2
frozenset({30248.0, 15066.0, 10060.0, 15094.0}) 2
frozenset({14177.0, 15094.0, 30248.0, 15066.0, 10060.0}) 2
frozenset({10086.0, 15094.0, 30248.0, 15066.0, 10060.0}) 2
frozenset({14177.0, 10086.0, 15094.0, 30248.0, 15066.0, 10060.0}) 2
frozenset({15066.0, 15052.0, 10060.0, 15094.0}) 2
frozenset({15094.0, 10060.0, 30248.0, 15066.0, 15052.0})

frozenset({30248.0, 14177.0, 10060.0, 10007.0}) 2
frozenset({14177.0, 15039.0, 10007.0}) 2
frozenset({14177.0, 15009.0}) 2
frozenset({14177.0, 15009.0, 15631.0}) 2
frozenset({14177.0, 15009.0, 15094.0}) 2
frozenset({14177.0, 15009.0, 15094.0, 15631.0}) 2
frozenset({14177.0, 15009.0, 15039.0}) 2
frozenset({14177.0, 15009.0, 15094.0, 15039.0}) 2
frozenset({14177.0, 15009.0, 15631.0, 15039.0}) 2
frozenset({14177.0, 15009.0, 15039.0, 15094.0, 15631.0}) 2
frozenset({14177.0, 10086.0}) 2
frozenset({30248.0, 14177.0, 10086.0}) 2
frozenset({14177.0, 10060.0, 10086.0}) 2
frozenset({30248.0, 14177.0, 10060.0, 10086.0}) 2
frozenset({14177.0, 15052.0, 10086.0}) 2
frozenset({30248.0, 14177.0, 15052.0, 10086.0}) 2
frozenset({14177.0, 15094.0, 10086.0}) 2
frozenset({14177.0, 10086.0, 15039.0}) 2
frozenset({14177.0, 14196.0}) 3
frozenset({30248.0, 14177.0, 14196.0}) 2
frozenset({14177.0, 15052.0, 14196.0}) 2
frozenset({14177.0, 14196.0, 10060.0}) 2
frozenset({30248.0, 14177.0, 14196.0, 10060.0}) 2
fro

...

frozenset({30248.0, 11114.0, 15012.0}) 5
frozenset({30248.0, 11114.0}) 11
frozenset({14281.0, 10068.0}) 4
frozenset({30248.0, 14281.0, 10068.0}) 4
frozenset({14322.0, 10068.0}) 4
frozenset({15052.0, 10068.0}) 4
frozenset({30248.0, 15052.0, 10068.0}) 4
frozenset({10068.0, 15631.0}) 4
frozenset({14475.0, 10068.0}) 4
frozenset({10068.0, 10086.0}) 5
frozenset({30248.0, 10068.0, 10086.0}) 5
frozenset({14442.0, 10068.0}) 5
frozenset({30248.0, 14442.0, 10068.0}) 4
frozenset({14442.0, 10068.0, 10086.0}) 4
frozenset({30248.0, 14442.0, 10068.0, 10086.0}) 4
frozenset({15012.0, 10068.0}) 6
frozenset({30248.0, 15012.0, 10068.0}) 4
frozenset({10068.0, 15094.0}) 6
frozenset({30248.0, 10068.0, 15094.0}) 6
frozenset({10068.0, 14622.0}) 7
frozenset({30248.0, 10068.0, 14622.0}) 6
frozenset({30248.0, 10068.0}) 13
frozenset({14225.0, 14322.0}) 4
frozenset({14225.0, 14319.0}) 4
frozenset({14225.0, 30170.0}) 4
frozenset({30248.0, 14225.0, 30170.0}) 4
frozenset({14225.0, 15009.0}) 4
frozenset({14225.0, 15039.

frozenset({30248.0, 15012.0, 15066.0, 15052.0}) 7
frozenset({15066.0, 15052.0, 15094.0}) 8
frozenset({30248.0, 15066.0, 15052.0, 15094.0}) 7
frozenset({30248.0, 15066.0, 15052.0}) 11
frozenset({15066.0, 15039.0}) 14
frozenset({15066.0, 15094.0, 15039.0}) 9
frozenset({15066.0, 15012.0, 15094.0, 15039.0}) 6
frozenset({15012.0, 15094.0, 30248.0, 15066.0, 15039.0}) 5
frozenset({30248.0, 15066.0, 15094.0, 15039.0}) 7
frozenset({15066.0, 15012.0, 15039.0}) 9
frozenset({30248.0, 15066.0, 15012.0, 15039.0}) 8
frozenset({30248.0, 15066.0, 15039.0}) 12
frozenset({15066.0, 15012.0}) 16
frozenset({15066.0, 15012.0, 15094.0}) 9
frozenset({30248.0, 15066.0, 15012.0, 15094.0}) 8
frozenset({30248.0, 15066.0, 15012.0}) 15
frozenset({15066.0, 15094.0}) 18
frozenset({30248.0, 15066.0, 15094.0}) 14
frozenset({30248.0, 15066.0}) 27
frozenset({30170.0, 14442.0}) 4
frozenset({14442.0, 15631.0}) 4
frozenset({14442.0, 14366.0}) 4
frozenset({30248.0, 14442.0, 14366.0}) 4
frozenset({15009.0, 14442.0}) 4
frozense

frozenset({10064.0, 10060.0}) 6
frozenset({10064.0, 30248.0}) 7
frozenset({15052.0, 14189.0}) 6
frozenset({15012.0, 14189.0}) 6
frozenset({30248.0, 14189.0}) 8
frozenset({14573.0, 15094.0}) 6
frozenset({30248.0, 14573.0}) 7
frozenset({15094.0, 34214.0}) 6
frozenset({30248.0, 34214.0}) 10
frozenset({30248.0, 14799.0}) 6
frozenset({14354.0, 15052.0}) 6
frozenset({14354.0, 14098.0}) 7
frozenset({30248.0, 14354.0}) 9
frozenset({30248.0, 14740.0}) 10
frozenset({30248.0, 14301.0}) 8
frozenset({14123.0, 15094.0}) 6
frozenset({30248.0, 14123.0, 15094.0}) 6
frozenset({14123.0, 15012.0}) 7
frozenset({30248.0, 14123.0, 15012.0}) 7
frozenset({30248.0, 14123.0}) 9
frozenset({30216.0, 30248.0}) 6
frozenset({30216.0, 30170.0}) 6
frozenset({30248.0, 14126.0}) 8
frozenset({10163.0, 15012.0}) 6
frozenset({30248.0, 10163.0, 15012.0}) 6
frozenset({30248.0, 10163.0}) 11
frozenset({30248.0, 14191.0}) 7
frozenset({15094.0, 14191.0}) 7
frozenset({30248.0, 11036.0}) 9
frozenset({31017.0, 15012.0}) 6
frozenset(

frozenset({15009.0, 15094.0}) 31
frozenset({30248.0, 15009.0, 15094.0}) 23
frozenset({30248.0, 15009.0}) 47
frozenset({15052.0, 15631.0}) 18
frozenset({15039.0, 15052.0, 15631.0}) 6
frozenset({15012.0, 15052.0, 15631.0}) 8
frozenset({30248.0, 15012.0, 15052.0, 15631.0}) 6
frozenset({15052.0, 15094.0, 15631.0}) 9
frozenset({30248.0, 15052.0, 15631.0}) 11
frozenset({15039.0, 15631.0}) 27
frozenset({15039.0, 15094.0, 15631.0}) 9
frozenset({30248.0, 15039.0, 15094.0, 15631.0}) 7
frozenset({15039.0, 15012.0, 15631.0}) 12
frozenset({30248.0, 15039.0, 15012.0, 15631.0}) 10
frozenset({30248.0, 15039.0, 15631.0}) 18
frozenset({15012.0, 15631.0}) 32
frozenset({15012.0, 15094.0, 15631.0}) 15
frozenset({30248.0, 15012.0, 15094.0, 15631.0}) 12
frozenset({30248.0, 15012.0, 15631.0}) 22
frozenset({15094.0, 15631.0}) 33
frozenset({30248.0, 15094.0, 15631.0}) 20
frozenset({30248.0, 15631.0}) 49
frozenset({15052.0, 15039.0}) 31
frozenset({15012.0, 15052.0, 15039.0}) 12
frozenset({15012.0, 15052.0, 15094

frozenset({30248.0, 15012.0, 15094.0}) 51
frozenset({30248.0, 15012.0}) 102
frozenset({30248.0, 15094.0}) 107
for bndno, old_data: 1.4574502600442134
for bndno, new_data: 10.671083403481418
